# Librerías

In [1]:
# pip install geopy

In [2]:
# pip install meteostat

In [3]:
import json
import time
import uuid
import random
from datetime import datetime, timedelta
import pytz
from geopy.geocoders import Nominatim
from meteostat import Point, Hourly
# employees = {i: (f"{i+1}", f"{i+101}") for i in range(100)}

In [4]:
# Definir las coordenadas aproximadas de Medellín
lat_range = (6.217, 6.317)  # Latitud de Medellín
lon_range = (-75.567, -75.467)  # Longitud de Medellín

# Inicializar el geolocalizador
geolocator = Nominatim(user_agent="medellin_locator")

# Obtención de datos

In [5]:
# Cargar puntos de venta válidos desde el archivo JSON
with open('valid_points.json', 'r') as file:
    points_of_sale = json.load(file)

# Generar 200 empleados, 2 por cada punto de venta
employees = {i: (f"Employee_{i*2+1}", f"Employee_{i*2+2}") for i in range(100)}

# Inicializar el geolocalizador
geolocator = Nominatim(user_agent="medellin_locator")

# Función para obtener la temperatura actual
def get_current_temperature(latitude, longitude):
    location = Point(latitude, longitude)
    start = datetime.now() - timedelta(hours=1)
    end = datetime.now()
    data = Hourly(location, start, end)
    data = data.fetch()
    temperature = data['temp'].mean() if not data.empty else 20  # Usar 20 como valor predeterminado si no hay datos
    return round(temperature, 1)

# Generar cantidad de productos en función de la temperatura
def generate_quantity_products(temperature):
    # Normalizar la temperatura entre un rango de 0 y 1
    temp_min, temp_max = 18, 24  # Rango de temperaturas en °C
    normalized_temp = (temperature - temp_min) / (temp_max - temp_min)
    
    # Generar la cantidad de productos basada en la temperatura normalizada
    min_quantity, max_quantity = 1, 50
    mean_quantity = normalized_temp * (max_quantity - min_quantity) + min_quantity
    
    # Añadir variabilidad a la cantidad de productos
    quantity = int(random.gauss(mean_quantity,10))  # Usar una desviación estándar de 10
    
    # Asegurar que la cantidad esté dentro del rango permitido
    return max(min(quantity, max_quantity), min_quantity)

# Generar datos simulados
def generate_event(pos_index):
    pos = points_of_sale[pos_index]
    worker = employees[pos_index]
    date_now = datetime.now(pytz.timezone('America/Bogota'))
    temperature = get_current_temperature(pos["latitude"], pos["longitude"])
    quantity_products = generate_quantity_products(temperature)
    
    event = {
        "latitude": pos["latitude"],
        "longitude": pos["longitude"],
        "date": date_now.strftime("%d/%m/%Y %H:%M:%S"),
        "customer_id": random.randint(1000, 2000),
        "employee_id": worker[0] if date_now.weekday() < 5 else worker[1],  # Lunes a Viernes / Fines de Semana
        "quantity_products": quantity_products,
        "order_id": str(uuid.uuid4()),
        "commune": pos["commune"],
        "neighborhood": pos["neighborhood"],
        "partition_date": date_now.strftime("%d%m%Y"),
        "event_date": date_now.strftime("%d/%m/%Y %H:%M:%S"),
        "event_day": date_now.day,
        "event_hour": date_now.hour,
        "event_minute": date_now.minute,
        "event_month": date_now.month,
        "event_second": date_now.second,
        "event_year": date_now.year,
        "current_temperature": temperature
    }
    return event


# Enviar los datos
def send_data(s, conn, data):
    conn.sendall(json.dumps(data).encode() + b'\n')
    print("Enviado:", data)

# Socket

In [6]:
import socket

In [ ]:
# Socket
def servidor():
    
    host = 'localhost'  # Nombre del host o dirección IP
    port = 2222  # Puerto en el que escucha el servidor

    s = socket.socket()
    s.bind((host, port))
    s.listen(1)
    print(f"Listening on {host}:{port}")
    
    conn, addr = s.accept()
    print(f"Connected from {addr}")
    
    try:
        while True:
            event = generate_event(random.randint(0, 99))
            # Esperar 30 segundos antes de generar el siguiente evento
            send_data(s, conn, event)
            # Guardado en un archivo JSON
            with open('simulated_data_02.json', 'a') as file:
                file.write(json.dumps(event) + "\n")
            
            time.sleep(1)
            
    except KeyboardInterrupt:
        print("Stopping the connection ...")
        conn.close()
        s.close()
        print("Connection stopped.")
    
    except BrokenPipeError:
        print("The connection has been closed.")
        conn.close()
        s.close()

if __name__ == "__main__":
    servidor()

Listening on localhost:2222
Connected from ('127.0.0.1', 47086)
Enviado: {'latitude': 6.233554, 'longitude': -75.565071, 'date': '17/06/2024 17:42:09', 'customer_id': 1580, 'employee_id': 'Employee_129', 'quantity_products': 36, 'order_id': 'fe60082e-fdb9-487e-a093-f6f63f4e8719', 'commune': 'Comuna 10 - La Candelaria', 'neighborhood': 'Barrio San Diego', 'partition_date': '17062024', 'event_date': '17/06/2024 17:42:09', 'event_day': 17, 'event_hour': 17, 'event_minute': 42, 'event_month': 6, 'event_second': 9, 'event_year': 2024, 'current_temperature': 24.0}
Enviado: {'latitude': 6.226864, 'longitude': -75.52891, 'date': '17/06/2024 17:42:11', 'customer_id': 1353, 'employee_id': 'Employee_65', 'quantity_products': 27, 'order_id': 'f0744cfa-2cfd-493d-9d82-0903023ac36a', 'commune': 'Comuna 9 - Buenos Aires', 'neighborhood': 'Ocho de Marzo', 'partition_date': '17062024', 'event_date': '17/06/2024 17:42:11', 'event_day': 17, 'event_hour': 17, 'event_minute': 42, 'event_month': 6, 'event_se